In [8]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import jpholiday
import operator
import pandas as pd
import numpy as np

#ref. : https://qiita.com/hid_tanabe/items/3c5e6e85c6c65f7b38be
def isBizDay(DATE):
    Date = datetime.date(int(DATE.year), int(DATE.month), int(DATE.day))
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1

def n_bizdays_ago(n): #n営業日前を出す
    today = datetime.datetime.now()
    i = n
    while True:
        if isBizDay(today - timedelta(i)): #n日前が平日になるまで遡る
            n_bizdays = today - timedelta(i)
            break
        else:
            i+=1
    return n_bizdays

def stock_pl(df_purchase, df_company): #買った日付、買った銘柄（証券コード）,買った数　の列を含むdf
    pl_sum = 0
    df_purchase_pl = df_purchase #損益用に新しいdf
    #df_purchase_pl.insert(2, 'Company', '---')#会社名用
    df_purchase_pl['unit_buy'] = 0 #終値単価ー勝った時
    df_purchase_pl['unit_yes'] = 0 #終値単価ー昨日
    df_purchase_pl['buy'] = 0 #買った日の終値いくら　列追加
    df_purchase_pl['yesterday'] = 0 #昨日の終値いくら　列追加
    df_purchase_pl['P/L'] = 0 #昨日の終値時点での損益いくら　列追加
    #df_purchase_pl.loc['sum'] = 
    for i in range(len(df_purchase)):
        #df_purchase_pl['Company'][i] = df_company.query('コード == df_purchase["Code"][i]')['銘柄名'][0]
        #stooqのデータフレームの順番は、後の日付→→前の日付　買った日→昨日とするなら、昨日が一番上に来る。勝った日は一番下。
        df_purchase_buy2yest = data.DataReader(df_purchase_pl['Code'][i].astype('str') + '.JP', 'stooq', datetime.datetime.strptime(df_purchase['Date'][i], "%Y-%m-%d"), n_bizdays_ago(1))
        df_purchase_pl['unit_buy'][i] = df_purchase_buy2yest.iloc[-1]['Close']
        df_purchase_pl['unit_yes'][i] = df_purchase_buy2yest['Close'][0]
        df_purchase_pl['buy'][i] = df_purchase_buy2yest.iloc[-1]['Close'] * df_purchase_pl['Amount'][i]
        df_purchase_pl['yesterday'][i] = df_purchase_buy2yest['Close'][0] * df_purchase_pl['Amount'][i]
        df_purchase_pl['P/L'][i] = df_purchase_pl['yesterday'][i] - df_purchase_pl['buy'][i]
        pl_sum += df_purchase_pl['P/L'][i]

    print(df_purchase_pl)
    print(pl_sum)

def stock_bought_rank(df_purchase):
    twodays_ago = n_bizdays_ago(2)
    yesterday = n_bizdays_ago(1)
    df_ranking = pd.DataFrame(np.arange(4*len(df_purchase)).reshape(len(df_purchase), 4))#ランキング用のdf
    #df_ranking.set_axis(['Code', '2days_ago', '1day_ago', 'Diff'], axis=1)#列名をセット→失敗
    df_ranking.columns = ['Code', '2days_ago', '1day_ago', 'Diff']
    for i in range(len(df_purchase)):
        df_ranking['Code'][i] = df_purchase['Code'][i]
        df_temp = data.DataReader(df_ranking['Code'][i].astype('str') + '.JP', 'stooq', twodays_ago, yesterday)
        df_ranking['2days_ago'][i] = df_temp['Close'][1]#2営業日前の終値
        df_ranking['1day_ago'][i] = df_temp['Close'][0]#1営業日前の終値
        df_ranking['Diff'][i] = df_temp['Close'][0] - df_temp['Close'][1]#↑2つの差

    df_sorted = df_ranking.sort_values('Diff', ascending=False)#ソート(降順)
    print(df_sorted)


df1 = pd.read_csv('stock_purchase_data.csv', header=0)
df2 = pd.read_csv('data_j.csv', header=0)
stock_pl(df1, df2)
stock_bought_rank(df1)



#株の単価も表中に欲しい
#stooqは単に日本での証券コードに.jpをつけたものと違う??←要確認

UndefinedVariableError: name 'df_purchase' is not defined

In [11]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import pandas as pd

df1 = pd.read_csv('stock_purchase_data.csv', header=0)
#df1.set_index(['Date'])
print(df1)
print(df1['Date'][0])
print(df1.tail(1)['Code'])
print(df1.iloc[-1]['Code'])
print(len(df1))


         Date  Code  Amount
0  2023-05-15  7751     200
1  2023-05-15  9021     100
2  2023-05-15  9020     100
3  2023-05-15  7974     200
2023-05-15
3    7974
Name: Code, dtype: int64
7974
4
